In [1]:
#import packages
import os
import pandas as pd # data frame handling
import numpy as np # general package for data analysis
from scipy.io import loadmat #for matlab import to python
import matplotlib.pyplot as plt #figures
from sklearn.decomposition import PCA

### Selecting the participant 
#### Identify ID and number of states of the selected data

In [2]:
# In order to run analysis on a new participant data
# need to change the participant ID (ex:006PD) in 'path' for new analysis

path = "/Users/biapt/Desktop/Dimensionality Reduction/TR_organized/006PD/Time-resolved wPLI/alpha"
dirs = sorted(os.listdir(path))

states = []
states_num = []
ID = ''
frequencies = ['alpha', 'theta']
#frequencies = ['alpha']
pli = ['wPLI','dPLI']

for file in dirs:
    state = file[20:26]
    states.append(state)
    states_numbered = "state"+ str(len(states)) # start at state1
    states_num.append(states_numbered)
    ID = file[14:19]

if ID =="006PD": #"003PD"
    states.pop(0)
    states_num.pop(0)
    
print("Subject ID: ", ID)
print("\nTotal number of states: ", len(states))
print("\nTotal states:", states)
print("\nStates used in the analysis:", states_num)

Subject ID:  006PD

Total number of states:  2

Total states: ['midaz8', 'sedof2']

States used in the analysis: ['state2', 'state3']


## Time-resolved wpli

In [3]:
# creating a folder for each subject inside 'TR_pipeline_result' folder 
    # TR_pipeline_result -> subject code -> Time-resolved wpli -> alpha or theta

fig_file_directory_1 = '/Users/biapt/Desktop/Dimensionality Reduction/TR_pipeline_result/'
os.mkdir(os.path.join(fig_file_directory_1,ID)) # create a new folder in the results folder with ID

fig_file_directory_main = fig_file_directory_1 + ID
os.mkdir(os.path.join(fig_file_directory_main,'Time-resolved wPLI'))
#os.mkdir(os.path.join(fig_file_directory_1,ID,'Time-resolved dPLI'))

# open a text file to save information about the dimensions of the time-resolved data
file = open(fig_file_directory_main+'/'+ID+'_info.txt', 'w')
file.write('Dimensionality Reduction Pipeline Info for: ' + ID)
file.write('\n\n This only includes continuous infusion data 8 compared to baseline for 006PD')
file.write('\n\n< Time-resolved wPLI >')

# Creating a color pallete for PCA 
color_pallete = ["red","blue","green", "yellow", "orange", "black","brown","cyan", "pink", "darkgreen", "grey"]

for frequency in frequencies: # loop over alpha and theta frequencies
    file.write('\n\n' + frequency + '\n')
    fig_file_directory_2 = fig_file_directory_main+ '/' + 'Time-resolved wPLI'
    os.mkdir(os.path.join(fig_file_directory_2, frequency))# create a 'frequency folder' under ID in the results folder
    
    fig_file_directory_3 = fig_file_directory_2 + '/' + frequency +'/'
    time_points = [0]
    
    for i in range(len(states)): # loop over all the states in a given subject
        state = states[i] # name of each of the states (ex: Sedoff)
        state_num = states_num[i] # number of the state (ex: state1)

        directory_beginning = '/Users/biapt/Desktop/Dimensionality Reduction/TR_organized/' #beginning of the directory for organized fc mat. files
        # wpli fc data
        pli1 = 'wPLI' 
        pli2 = 'wpli'
        directory = directory_beginning+ID+'/Time-resolved '+pli1+'/'+frequency+'/'+pli2+'_TR_'+frequency+'_'+ID+'_'+state+'.mat' # complete directory for files
        state_step = loadmat(directory)
        data_state = state_step['result_wpli_TR'] # Match the state to state_number
        statement1 = 'The size of '+frequency+' '+pli1+' '+ ID +' '+ state +' is: '+ str(data_state.shape)
        print(statement1)
        file.write('\n'+statement1)
        
        # Determine the size of 2D array for each state
        time = data_state.shape[0] # length of the time stamp
        time_points.append(time)
        num_elect = data_state.shape[1] # number of electrodes

        # calculate the length of reduced electrode-electrode data
        # upper triangle of the data
        empty_array = np.zeros([num_elect, num_elect])
        half_data = empty_array[np.triu_indices(num_elect-1)]
        elect_connect = half_data.shape[0]

        # create 2D data for all the time stamps
        data_2d_state = np.zeros([time,elect_connect])

        for ii in range (time):
            data_at_time = data_state[ii,:]
            data_long = data_at_time[np.triu_indices(num_elect-1)]
            data_2d_state[ii] = data_long
    
        # plot 2D data of each state as a figure
        plt.imshow(data_2d_state, cmap = "jet")
        plt.clim(0, 0.25)
        plt.xlabel('Non-overlapping Electrode Connections')
        plt.ylabel('Time')
        plt.title('Time vs Electrode Connections: '+ ID +' '+ state +' ('+ frequency +')')
        fig_file_name = ID+'_'+frequency+'_TR_'+pli1+'_'+ state +'.png'
        plt.savefig(fig_file_directory_3+fig_file_name, bbox_inches ="tight",transparent = True)
        

        # combine the rest state and anes state data
        if i==0:
            data_combined= data_state
        else:
            data_combined = np.concatenate((data_combined, data_state))
    
        if i==len(states)-1:
            statement2 = '\nThe shape of the combined time-resolved '+pli1+' data in ' + frequency+ ' frequency is: '
            print(statement2,data_combined.shape,'\n')

            file.write(statement2 +' (')
            file.write(' '.join(str(s) for s in data_combined.shape) + ')\n')

            # 2D data calculation for the combined states
            time_2d = data_combined.shape[0] # length of the time stamp

            # 2D data
            data_2d_combined = np.zeros([time_2d,elect_connect])

            for iii in range (time_2d):
                data_at_time = data_combined[iii,:] #changed data_combined to data_2d_combined
                data_long = data_at_time[np.triu_indices(num_elect-1)]
                data_2d_combined[iii] = data_long
                
            plt.Figure()
            plt.imshow(data_2d_combined, cmap = "jet")
            plt.xlabel('Non-overlapping Electrode Connections')
            plt.ylabel('Time')
            plt.clim(0, 0.25)
            plt.title(ID +' '+frequency+'_combined_TR_'+pli1)
            fig_file_name2 = ID+'_'+frequency+'_combined_TR_'+pli1+'.png'
            plt.savefig(fig_file_directory_3+fig_file_name2,transparent = True)
            plt.close() #closing figure
            
            # PCA analysis
            pca = PCA(n_components=2)
            pca.fit(data_2d_combined)
            combined_pca = pca.transform(data_2d_combined)
            
            # print information about the transformed data
            print("original shape: ",data_2d_combined.shape)                                       
            print("transformed shape: ",combined_pca.shape)
            print("explained variance: ", pca.explained_variance_)
            print("explained variance ratio: ",pca.explained_variance_ratio_,'\n')
            
            # write on the file
            file.write('\nOriginal shape: '+' '.join(str(s) for s in data_2d_combined.shape))
            file.write('\nTransformed shape: '+' '.join(str(s) for s in combined_pca.shape))
            file.write('\nExplained variance: '+' '.join(str(s) for s in pca.explained_variance_))
            file.write('\nExplained variance ratio: '+ ' '.join(str(s) for s in pca.explained_variance_ratio_))
            
            plt.Figure() #opening new figure
            fig_file_name3 = 'PCA of '+ID+'_'+frequency+'_combined_TR_'+pli1+'.png'
            plt.xlabel('First Dimension')
            plt.ylabel('Second Dimension')
            plt.title('PCA Analysis: '+ID+' ('+frequency+')')
            
            #time stamp
            for time_point_num in range(len(time_points)-1):
                time1 = time_points[time_point_num]
                time2 = time1+time_points[time_point_num+1]
    
                plt.scatter(combined_pca[range(time1,time2),0], combined_pca[range(time1,time2),1], s=0.5, color= color_pallete[time_point_num])
            
            fig_file_name3 = 'PCA_'+ID+'_'+frequency+'_combined_TR_'+pli1+'.png'
            plt.savefig(fig_file_directory_3+fig_file_name3,transparent = True)
            plt.close()
            
            # plot the figure for the explained variance
            plt.figure()
            plt.plot(np.cumsum(pca.explained_variance_ratio_))
            plt.xlabel('number of components')
            plt.ylabel('cumulative explained variance')
            plt.title('Variance Explained by PCA: '+ID+'_'+frequency+'_'+pli1)
            fig_file_name4 = 'PCA_variance_explained.png'
            plt.savefig(fig_file_directory_3+fig_file_name4)
            plt.close()


The size of alpha wPLI 006PD midaz8 is: (288, 26, 26)
The size of alpha wPLI 006PD sedof2 is: (398, 26, 26)

The shape of the combined time-resolved wPLI data in alpha frequency is:  (686, 26, 26) 

original shape:  (686, 325)
transformed shape:  (686, 2)
explained variance:  [0.20482194 0.12533338]
explained variance ratio:  [0.09497295 0.05811526] 

The size of theta wPLI 006PD midaz8 is: (288, 26, 26)
The size of theta wPLI 006PD sedof2 is: (398, 26, 26)

The shape of the combined time-resolved wPLI data in theta frequency is:  (686, 26, 26) 

original shape:  (686, 325)
transformed shape:  (686, 2)
explained variance:  [0.43067081 0.25316793]
explained variance ratio:  [0.1193873  0.07018129] 



## Time-resolved dPLI

In [4]:
# creating a folder for each subject inside 'TR_pipeline_result' folder 
    # TR_pipeline_result -> subject code -> Time-resolved dpli -> alpha or theta

fig_file_directory_main = '/Users/biapt/Desktop/Dimensionality Reduction/TR_pipeline_result/'+ ID
os.mkdir(os.path.join(fig_file_directory_main,'Time-resolved dPLI'))

# Write on the text file to save information about the dimensions of the time-resolved data
file.write('\n\n< Time-resolved dPLI >')

# Creating a color pallete for PCA 
color_pallete = ["red","blue","green", "yellow", "orange", "black","brown","cyan", "pink", "darkgreen", "grey"]

for frequency in frequencies: # loop over alpha and theta frequencies
    file.write('\n\n' + frequency + '\n')
    fig_file_directory_2 = fig_file_directory_main+ '/' + 'Time-resolved dPLI'
    os.mkdir(os.path.join(fig_file_directory_2, frequency))# create a 'frequency folder' under ID in the results folder
    
    fig_file_directory_3 = fig_file_directory_2 + '/' + frequency +'/'
    time_points = [0]
    
    for i in range(len(states)): # loop over all the states in a given subject
        state = states[i] # name of each of the states (ex: Sedoff)
        state_num = states_num[i] # number of the state (ex: state1)

        directory_beginning = '/Users/biapt/Desktop/Dimensionality Reduction/TR_organized/' #beginning of the directory for organized fc mat. files
        # wpli fc data
        pli1 = 'dPLI' 
        pli2 = 'dpli'
        directory = directory_beginning+ID+'/Time-resolved '+pli1+'/'+frequency+'/'+pli2+'_TR_'+frequency+'_'+ID+'_'+state+'.mat' # complete directory for files
        state_step = loadmat(directory)
        data_state = state_step['result_dpli_TR'] # Match the state to state_number
        statement1 = 'The size of '+frequency+' '+pli1+' '+ ID +' '+ state +' is: '+ str(data_state.shape)
        print(statement1)
        file.write('\n'+statement1)
        
        # Determine the size of 2D array for each state
        time = data_state.shape[0] # length of the time stamp
        time_points.append(time)
        num_elect = data_state.shape[1] # number of electrodes

        # calculate the length of reduced electrode-electrode data
        # upper triangle of the data
        empty_array = np.zeros([num_elect, num_elect])
        half_data = empty_array[np.triu_indices(num_elect-1)]
        elect_connect = half_data.shape[0]

        # create 2D data for all the time stamps
        data_2d_state = np.zeros([time,elect_connect])

        for ii in range (time):
            data_at_time = data_state[ii,:]
            data_long = data_at_time[np.triu_indices(num_elect-1)]
            data_2d_state[ii] = data_long
    
        # plot 2D data of each state as a figure
        plt.imshow(data_2d_state, cmap = "jet")
        plt.clim(0.4, 0.7)
        plt.xlabel('Non-overlapping Electrode Connections')
        plt.ylabel('Time')
        plt.title('Time vs Electrode Connections: '+ ID +' '+ state +' ('+ frequency +')')
        fig_file_name = ID+'_'+frequency+'_TR_'+pli1+'_'+ state +'.png'
        plt.savefig(fig_file_directory_3+fig_file_name, bbox_inches ="tight",transparent = True)

        # combine the rest state and anes state data
        if i==0:
            data_combined= data_state
        else:
            data_combined = np.concatenate((data_combined, data_state))
    
        if i==len(states)-1:
            statement2 = '\nThe shape of the combined time-resolved '+pli1+' data in ' + frequency+ ' frequency is: '
            print(statement2,data_combined.shape,'\n')

            file.write(statement2 +' (')
            file.write(' '.join(str(s) for s in data_combined.shape) + ')\n')

            # 2D data calculation for the combined states
            time_2d = data_combined.shape[0] # length of the time stamp

            # 2D data
            data_2d_combined = np.zeros([time_2d,elect_connect])

            for iii in range (time_2d):
                data_at_time = data_combined[iii,:] #changed data_combined to data_2d_combined
                data_long = data_at_time[np.triu_indices(num_elect-1)]
                data_2d_combined[iii] = data_long
                
            plt.Figure()
            plt.imshow(data_2d_combined, cmap = "jet")
            plt.xlabel('Non-overlapping Electrode Connections')
            plt.ylabel('Time')
            plt.clim(0.4, 0.7)
            plt.title(ID +' '+frequency+'_combined_TR_'+pli1)
            fig_file_name2 = ID+'_'+frequency+'_combined_TR_'+pli1+'.png'
            plt.savefig(fig_file_directory_3+fig_file_name2,transparent = True)
            plt.close() #closing figure
            
            # PCA analysis
            pca = PCA(n_components=2)
            pca.fit(data_2d_combined)
            combined_pca = pca.transform(data_2d_combined)
            
            # print information about the transformed data
            print("original shape: ",data_2d_combined.shape)                                       
            print("transformed shape: ",combined_pca.shape)
            print("explained variance: ", pca.explained_variance_)
            print("explained variance ratio: ",pca.explained_variance_ratio_,'\n')
            
            # write on the file
            file.write('\nOriginal shape: '+' '.join(str(s) for s in data_2d_combined.shape))
            file.write('\nTransformed shape: '+' '.join(str(s) for s in combined_pca.shape))
            file.write('\nExplained variance: '+' '.join(str(s) for s in pca.explained_variance_))
            file.write('\nExplained variance ratio: '+ ' '.join(str(s) for s in pca.explained_variance_ratio_))
            
            fig_file_name3 = 'PCA of '+ID+'_'+frequency+'_combined_TR_'+pli1+'.png'
            plt.Figure() #opening new figure
            plt.xlabel('First Dimension')
            plt.ylabel('Second Dimension')
            plt.title('PCA Analysis: '+ID+' ('+frequency+')')
            
            #time stamp
            for time_point_num in range(len(time_points)-1):
                time1 = time_points[time_point_num]
                time2 = time1+time_points[time_point_num+1]
    
                plt.scatter(combined_pca[range(time1,time2),0], combined_pca[range(time1,time2),1], s=0.5, color= color_pallete[time_point_num])
            
            fig_file_name3 = 'PCA_'+ID+'_'+frequency+'_combined_TR_'+pli1+'.png'
            plt.savefig(fig_file_directory_3+fig_file_name3,transparent = True)
            plt.close()
            
            # plot the figure for the explained variance
            plt.figure()
            plt.plot(np.cumsum(pca.explained_variance_ratio_))
            plt.xlabel('number of components')
            plt.ylabel('cumulative explained variance')
            plt.title('Variance Explained by PCA: '+ID+'_'+frequency+'_'+pli1)
            fig_file_name4 = 'PCA_variance_explained.png'
            plt.savefig(fig_file_directory_3+fig_file_name4)
            plt.close()
            
file.close() 


The size of alpha dPLI 006PD midaz8 is: (288, 26, 26)
The size of alpha dPLI 006PD sedof2 is: (398, 26, 26)

The shape of the combined time-resolved dPLI data in alpha frequency is:  (686, 26, 26) 

original shape:  (686, 325)
transformed shape:  (686, 2)
explained variance:  [0.08767528 0.06512292]
explained variance ratio:  [0.11810821 0.08772771] 

The size of theta dPLI 006PD midaz8 is: (288, 26, 26)
The size of theta dPLI 006PD sedof2 is: (398, 26, 26)

The shape of the combined time-resolved dPLI data in theta frequency is:  (686, 26, 26) 

original shape:  (686, 325)
transformed shape:  (686, 2)
explained variance:  [0.21767633 0.11609492]
explained variance ratio:  [0.17419357 0.09290393] 

